In [2]:
import pyzx as zx
import random
from qiskit import QuantumCircuit
from qiskit.circuit import QuantumCircuit, ParameterVector
import csv
import time

In [3]:
def generate_H_S_CNOT_T_circuit(qubits, gates, p_t=0, seed=1000):
    random.seed(seed)  
    p_s = 0.333 * (1.0 - p_t)  
    p_had = 0.333 * (1.0 - p_t)  
    p_cnot = 0.333 * (1.0 - p_t)  

    c = zx.Circuit(qubits) 
    for _ in range(gates):
        r = random.random() 
        if r < p_had:
            c.add_gate("HAD", random.randrange(qubits))
        elif r < p_had + p_s:
            c.add_gate("S", random.randrange(qubits))
        elif r < p_had + p_s + p_t:
            c.add_gate("T", random.randrange(qubits))
        else:
            tgt = random.randrange(qubits)
            while True:
                ctrl = random.randrange(qubits)
                if ctrl != tgt:
                    break
            c.add_gate("CNOT", tgt, ctrl)
    return c

In [57]:
qubit = 8
gate_num = 30
p_t = 0.3
seed = 1000

print("start basci")
c = generate_H_S_CNOT_T_circuit(qubits = qubit,gates= gate_num,p_t = p_t,seed = seed)
c = zx.optimize.basic_optimization(c)


print(c.stats())
zx.draw(c)
print("start clifford")
g = c.to_graph()
zx.draw(g)

zx.simplify.spider_simp(g)
zx.draw(g)

zx.simplify.to_gh(g)
zx.draw(g)


zx.simplify.clifford_simp(g)
zx.draw(g)
print("start extract")
#抽出
c = zx.extract.streaming_extract(g)
zx.draw(c)
print(c.stats())
c = zx.optimize.basic_optimization(c)
zx.draw(c)
print(c.stats())

#qasm_str_aft = c.to_basic_gates().to_qasm()
#circuit_qiskit_aft = QuantumCircuit.from_qasm_str(qasm_str_aft)
#circuit_qiskit_aft.draw('mpl')

start basci
Circuit  on 8 qubits with 21 gates.
        5 is the T-count
        16 Cliffords among which
        7 2-qubit gates (6 CNOT, 1 other) and
        6 Hadamard gates.


start clifford


spider_simp: 4. 2. 1.  3 iterations


start extract
This function is deprecated. Call extract_circuit() instead.


Circuit  on 8 qubits with 42 gates.
        5 is the T-count
        37 Cliffords among which
        16 2-qubit gates (0 CNOT, 16 other) and
        15 Hadamard gates.


Circuit  on 8 qubits with 36 gates.
        5 is the T-count
        31 Cliffords among which
        16 2-qubit gates (15 CNOT, 1 other) and
        10 Hadamard gates.


In [33]:


def split_circ_on_T(c):
    """Produces a list of circuits whose odd elements are T-free circuits and
    whose even elements are circuits consisting only of T gates. Note it expects
    the circuit to only have basic gates (i.e. at most 1 control)."""
    q = c.qubits
    
    # keep lists of T-free and T-only circuits
    cs0 = [zx.Circuit(q)]
    cs1 = [zx.Circuit(q)]
    after = zx.Circuit(q)
    blocked = set()
    blocked2 = set()
    for g in c.gates:
        #print(g)
        #gのphaseがpi/4の倍数の場合
        #if g.phase % (zx.pi /2 ) != 0 and g.phase % (zx.pi / 4)  == 0:
        if g.name == 'T':
            if g.target in blocked2:
                cs0.append(after)
                after = zx.Circuit(q)
                cs1.append(zx.Circuit(q))
                blocked.clear()
                blocked2.clear()
                cs1[-1].gates.append(g)
                blocked.add(g.target)
            else:
                cs1[-1].gates.append(g)
                blocked.add(g.target)
        else:
            if g.name in ('CNOT','HAD') and g.target in blocked: 
                after.gates.append(g)
                blocked2.add(g.target)
                if g.name == 'CNOT': 
                    blocked.add(g.control)
            elif g.target in blocked2 or (g.name == 'CNOT' and g.control in blocked2):
                after.gates.append(g)
                if g.name == 'CNOT': 
                    blocked.add(g.target)
                    blocked2.add(g.target)
            else:
                cs0[-1].gates.append(g)
    cs = []
    for i,c0 in enumerate(cs0):
        cs.append(c0)
        cs.append(cs1[i])
    cs.append(after)
    return cs
def merge_circ(cs):
    c0 = zx.Circuit(cs[0].qubits)
    for c in cs:
        c0.add_circuit(c)
    return c0




c = generate_H_S_CNOT_T_circuit(qubits = 2,gates= 100,p_t = 0.4,seed = seed)
c = zx.optimize.basic_optimization(c)

zx.draw(c)
print(c.tcount())
# 分割
circuits = split_circ_on_T(c)
print("分割された回路:", circuits)
for i in range(len(circuits)):
    print("回路", i, ":", circuits[i])
    print("統計情報:", circuits[i].stats())
    zx.draw(circuits[i])
    print("\n")

# 再結合
merged_circuit = merge_circ(circuits)
print("再結合された回路:", merged_circuit)

7
分割された回路: [Circuit(2 qubits, 0 bits, 19 gates), Circuit(2 qubits, 0 bits, 0 gates), Circuit(2 qubits, 0 bits, 0 gates)]
回路 0 : Circuit(2 qubits, 0 bits, 19 gates)
統計情報: Circuit  on 2 qubits with 19 gates.
        7 is the T-count
        12 Cliffords among which
        4 2-qubit gates (2 CNOT, 2 other) and
        6 Hadamard gates.




回路 1 : Circuit(2 qubits, 0 bits, 0 gates)
統計情報: Circuit  on 2 qubits with 0 gates.
        0 is the T-count
        0 Cliffords among which
        0 2-qubit gates (0 CNOT, 0 other) and
        0 Hadamard gates.




回路 2 : Circuit(2 qubits, 0 bits, 0 gates)
統計情報: Circuit  on 2 qubits with 0 gates.
        0 is the T-count
        0 Cliffords among which
        0 2-qubit gates (0 CNOT, 0 other) and
        0 Hadamard gates.




再結合された回路: Circuit(2 qubits, 0 bits, 19 gates)
